# Introduction/Abstract

Data set from King County Washington.

# Packages Needed/Setting Global Variables

In [11]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import ols


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [12]:
def Decade(List):
    decade_list=[]
    oldest = List.min()
    i=0
    for x in List:
        i += 1
        while len(decade_list) < i:
            if (x >= oldest) & (x < (oldest+10)):
                decade_list.append('{}-{}'.format(oldest,(oldest+10)))
            oldest += 10
        oldest = List.min()
    return decade_list

In [13]:
def zoningLat(List):
    lowest = List.min()
    highest = List.max()
    distance = highest - lowest
    distance = math.ceil(distance*69)
    zones = []
    for entry in List:
        for zone in list(range(1,distance+1)):
            if ((lowest+(zone-1)/69) <= entry) &(entry < (lowest + zone/69)):
                zones.append(zone)
    return zones

def zoningLong(List):
    lowest = List.min()
    highest = List.max()
    distance = highest-lowest
    distance = math.ceil(distance*54.6)
    zones = []
    for entry in List:
        for zone in list(range(1,distance+1)):
            if ((lowest+(zone-1)/54.6) <= entry) & (entry < (lowest + zone/54.6)):
                zones.append(zone)
    return zones

In [14]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

# EDA

## Questions about the Data

## Import and Clean the Data

In [15]:
# Import the dataset
df = pd.read_csv('kc_house_data.csv')

#Dropping id column
df = df.drop(['id'],axis=1)

# Creating a new column for the year sold, dropping origianl date
df['year_sold'] = df.date.apply(lambda x: x[-4:]).astype(int)
df = df.drop(['date'], axis=1)

# Changing bedroom number for outlier, I think it was a typo of hitting 3 twice
# based off of the sqft
df.at[15856, 'bedrooms'] = 3

# Replacing NaN values in 'waterfront' with 0
df['waterfront'] = df['waterfront'].fillna(0)

# Replacing NaN values in view with the median/mode value which is 0 (63 entries)
# 0 means house was not viewed
df['view'] = df['view'].fillna(0)

# Replacing ? in sqft_basement and converting to float
df['sqft_basement'] = df['sqft_basement'].replace('?',0).astype(float)

# yr_renovated and renovated columns
df['yr_renovated'] = df['yr_renovated'].fillna(0)
df['Renovated'] = df['yr_renovated'] > 0
df['Renovated'] = df['Renovated'].astype(int)
df['yr_renovated'] = df['yr_renovated'].replace(0,df['yr_built'])

#Adding column for age when sold
df['Age_When_Sold'] = df['year_sold'] - df['yr_built']

#Adding column for years since renovation when sold and dropping yr_renovated
df['Age_Since_Renovation'] = df['year_sold'] - df['yr_renovated']

#Adding interaction variable Renovated*Age since renovation
df['Renovated*Age'] = df['Renovated'] * df['Age_Since_Renovation']

#Adding zones for Lat and Long
df['Lat_Zones'] = zoningLat(df['lat'])
df['Long_Zones'] = zoningLong(df['long'])
df['Lat_Zones'] = df['Lat_Zones'].astype(str)
df['Long_Zones'] = df['Long_Zones'].astype(str)
#df['Zones'] = df['Lat_Zones'] + 'x' + df['Long_Zones']
df = df.drop(['lat','long','Long_Zones','Lat_Zones'],axis=1)

#Grouping year built by decade
df['yr_built'] = Decade(df.yr_built)

#Correlations that are getting dropped
df = df.drop(['yr_renovated'],axis=1)
df = df.drop(['Age_Since_Renovation'],axis=1)
df = df.drop(['sqft_living'],axis=1)
df = df.drop(['Renovated*Age'],axis=1)

In [16]:
#Creating Dummie Variables double check this list
dummy_list = ['bedrooms', 'bathrooms', 'floors', 'view', 'condition',
                'grade', 'yr_built', 'zipcode', 'year_sold',
                ] #Add 'Zones' back in later
dum=df[dummy_list].astype('category')
dummies = pd.get_dummies(dum[dummy_list],prefix=dummy_list,drop_first=True)
df_preprocessed = df.drop(dummy_list, axis=1)
df_preprocessed = pd.concat([df_preprocessed,dummies],axis=1)

## Notes

# Modeling

In [17]:
#Omitting zones for now, too much data to process
y = df_preprocessed['price']
X = df_preprocessed.drop(['price'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
X_train_int = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_int).fit()
model.summary()
#Adj. R-squared values
#First run 0.853
#Second run 0.850 (after taking out yr_renovated)
#Third run 0.857 (after taking out Age_Since_Renovation)
#Fourth run 0.851 (after taking out sqft_living)
#Fifth run 0.851 (after taking out Renovated*Age)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     640.0
Date:                Mon, 19 Oct 2020   Prob (F-statistic):               0.00
Time:                        20:26:54   Log-Likelihood:            -2.2978e+05
No. Observations:               17277   AIC:                         4.599e+05
Df Residuals:                   17127   BIC:                         4.610e+05
Df Model:                         149                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               8.544e+04   8.36e+04      1.022      0.307   -7.84e+04    2.49e+05
sqft_lot               0.2415      0.039      6.267      0.000       0.166       0.317
waterfront          6.154e+05   1.71e+04     36.005      0.000    5.82e+05    6.49e+05
sqft_above           153.7956      3.269     47.040      0.000     147.387     160.204
sqft_basement        111.3924      3.662     30.422      0.000     104.215     118.569
sqft_living15         22.8378      2.996      7.624      0.000      16.966      28.709
sqft_lot15            -0.0200      0.064     -0.314      0.754      -0.145       0.105
Renovated           6.236e+04   6613.587      9.429      0.000    4.94e+04    7.53e+04
Age_When_Sold        -89.5187    400.019     -0.224      0.823    -873.597     694.559
bedrooms_2          1.036e+04   1.26e+04      0.822      0.411   -1.43e+04    3.51e+04
bedrooms_3          2.273e+04   1.26e+04      1.798      0.072   -2045.291    4.75e+04
bedrooms_4          8357.7023   1.29e+04      0.646      0.518    -1.7e+04    3.37e+04
bedrooms_5         -6346.8354   1.36e+04     -0.466      0.641   -3.31e+04    2.04e+04
bedrooms_6         -4.429e+04   1.66e+04     -2.660      0.008   -7.69e+04   -1.17e+04
bedrooms_7         -6.504e+04   2.92e+04     -2.231      0.026   -1.22e+05   -7888.865
bedrooms_8         -4.593e+04   4.65e+04     -0.988      0.323   -1.37e+05    4.52e+04
bedrooms_9         -2.146e+05   7.51e+04     -2.859      0.004   -3.62e+05   -6.75e+04
bedrooms_10        -3.031e+05   8.62e+04     -3.515      0.000   -4.72e+05   -1.34e+05
bedrooms_11        -5.616e+04   1.46e+05     -0.384      0.701   -3.43e+05    2.31e+05
bathrooms_0.75     -1.193e+04   7.57e+04     -0.158      0.875    -1.6e+05    1.36e+05
bathrooms_1.0       2.273e+04   7.28e+04      0.312      0.755    -1.2e+05    1.65e+05
bathrooms_1.25     -6.125e+04   8.75e+04     -0.700      0.484   -2.33e+05     1.1e+05
bathrooms_1.5       1.849e+04    7.3e+04      0.253      0.800   -1.25e+05    1.61e+05
bathrooms_1.75      1.402e+04   7.29e+04      0.192      0.848   -1.29e+05    1.57e+05
bathrooms_2.0       1.611e+04   7.29e+04      0.221      0.825   -1.27e+05    1.59e+05
bathrooms_2.25      3.486e+04    7.3e+04      0.478      0.633   -1.08e+05    1.78e+05
bathrooms_2.5       2.339e+04   7.29e+04      0.321      0.748    -1.2e+05    1.66e+05
bathrooms_2.75      1.754e+04   7.31e+04      0.240      0.810   -1.26e+05    1.61e+05
bathrooms_3.0       4.104e+04   7.32e+04      0.561      0.575   -1.02e+05    1.84e+05
bathrooms_3.25      7.512e+04   7.33e+04      1.025      0.305   -6.85e+04    2.19e+05
bathrooms_3.5       3.508e+04   7.32e+04      0.479      0.632   -1.08e+05    1.79e+05
bathrooms_3.75      1.233e+05   7.42e+04      1.661      0.097   -2.22e+04    2.69e+05
bathrooms_4.0       1.207e+05   7.45e+

In [11]:
continuous = ['sqft_lot', 'sqft_above',
              'sqft_basement', 'sqft_living15', 'sqft_lot15','Age_When_Sold']

In [9]:
#Remove Correlated Features
test = X_train#[continuous]

In [10]:
test_corr=test.corr().abs().stack().reset_index().sort_values(0, ascending=False)
test_corr['pairs'] = list(zip(test_corr.level_0, test_corr.level_1))
test_corr.set_index(['pairs'],inplace=True)
test_corr.drop(['level_0','level_1'],axis=1,inplace=True)
test_corr.columns = ['cc']
test_corr.drop_duplicates(inplace=True)
test_corr[(test_corr.cc >.75) & (test_corr.cc<1)]

,cc
pairs,
"(condition_3, condition_4)",0.813554


In [17]:
#Looks like yr_built and yr_renovated is highly correlated, dropping yr_renovated column
#Looks like Age_When_sold and Age_Since_Renovation is highly correlated, dropping Age_Since_Renovation
#Looks like sqft_living and sqft_above is highly correlated, dropping sqft_living
#Looks like Renovated*Age and Renovated is highly correlated, dropping Renovated*Age
#Looks like sqft_living and sqft_living15 is highly correlated, dropping sqft_living
#Also looks like condition_3 and condition_4 is highly correlated, not going to do anything for these since they are from the both category
#Did this in EDA section

In [19]:
results = stepwise_selection(X_train,y_train)

/Users/stevenkyle/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


Add  grade_10                       with p-value 0.0
Add  sqft_living15                  with p-value 0.0
Add  sqft_above                     with p-value 0.0
Add  Age_When_Sold                  with p-value 0.0
Add  waterfront                     with p-value 0.0
Add  sqft_basement                  with p-value 0.0
Add  zipcode_98004                  with p-value 0.0
Add  zipcode_98039                  with p-value 3.23747e-196
Add  grade_12                       with p-value 9.56164e-139
Add  grade_11                       with p-value 5.81134e-186
Add  grade_13                       with p-value 4.7878e-180
Add  grade_9                        with p-value 1.91709e-178
Add  grade_8                        with p-value 1.92423e-187
Add  zipcode_98112                  with p-value 1.09651e-132
Add  zipcode_98040                  with p-value 2.75136e-125
Add  zipcode_98033                  with p-value 2.45308e-85
Add  zipcode_98023                  with p-value 5.37478e-81
Add  bathroo

In [26]:
TestFeatures=X_train[results]
TestFeatures.head()

,grade_10,sqft_living15,sqft_above,Age_When_Sold,waterfront,sqft_basement,zipcode_98004,zipcode_98039,grade_12,grade_11,grade_13,grade_9,grade_8,zipcode_98112,zipcode_98040,zipcode_98033,zipcode_98023,bathrooms_8.0,view_4.0,zipcode_98105,zipcode_98103,zipcode_98119,zipcode_98115,zipcode_98199,zipcode_98117,zipcode_98109,zipcode_98107,zipcode_98122,zipcode_98102,zipcode_98116,zipcode_98144,bathrooms_7.75,zipcode_98092,zipcode_98052,zipcode_98006,view_3.0,zipcode_98008,zipcode_98136,zipcode_98034,zipcode_98005,zipcode_98053,zipcode_98029,zipcode_98125,condition_5,zipcode_98177,zipcode_98126,year_sold_2015,zipcode_98007,zipcode_98133,view_2.0,zipcode_98027,zipcode_98074,zipcode_98118,zipcode_98155,zipcode_98075,zipcode_98072,view_1.0,bathrooms_4.75,bathrooms_5.5,yr_built_2010-2020,sqft_lot,Renovated,zipcode_98028,zipcode_98106,condition_4,zipcode_98011,bathrooms_6.0,bathrooms_4.25,bedrooms_3,bathrooms_5.0,bathrooms_6.25,zipcode_98042,bathrooms_3.75,zipcode_98108,floors_3.0,zipcode_98056,bathrooms_3.25,zipcode_98077,zipcode_98024,zipcode_98146,zipcode_98059,zipcode_98065,zipcode_98045,zipcode_98014,zipcode_98019,floors_2.0,bathrooms_4.0,bathrooms_5.25,floors_2.5,bedrooms_6,bathrooms_4.5,bathrooms_5.75,bedrooms_9,yr_built_1930-1940,bedrooms_10,zipcode_98022,zipcode_98003,bathrooms_2.25,bedrooms_5,bathrooms_3.0,zipcode_98166,yr_built_1920-1930,floors_1.5,bedrooms_7
337,0,1820,1510,103,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
17238,0,3670,3160,11,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5635,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7530,0,1420,870,114,0.0,640.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3240,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18431,0,2080,1820,23,0.0,0.0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,7750,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17469,0,1070,980,67,0.0,980.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8136,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
TestFeatures_X_int = sm.add_constant(TestFeatures)
model2 = sm.OLS(y_train, TestFeatures_X_int).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     911.6
Date:                Mon, 19 Oct 2020   Prob (F-statistic):               0.00
Time:                        20:47:23   Log-Likelihood:            -2.2984e+05
No. Observations:               17277   AIC:                         4.599e+05
Df Residuals:                   17172   BIC:                         4.607e+05
Df Model:                         104                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -8.235e+04   6757.939    -12.185      0.000   -9.56e+04   -6.91e+04
grade_10             2.14e+05   7169.536     29.848      0.000       2e+05    2.28e+05
sqft_living15         23.5475      2.937      8.017      0.000      17.790      29.305
sqft_above           155.2506      2.971     52.259      0.000     149.428     161.074
Age_When_Sold        226.4479     68.876      3.288      0.001      91.443     361.452
waterfront          6.112e+05   1.68e+04     36.282      0.000    5.78e+05    6.44e+05
sqft_basement        113.4504      3.214     35.299      0.000     107.151     119.750
zipcode_98004       7.561e+05   9953.706     75.963      0.000    7.37e+05    7.76e+05
zipcode_98039       1.089e+06   2.36e+04     46.222      0.000    1.04e+06    1.14e+06
grade_12             7.89e+05   1.96e+04     40.324      0.000    7.51e+05    8.27e+05
grade_11            4.108e+05    1.1e+04     37.481      0.000    3.89e+05    4.32e+05
grade_13             1.06e+06   5.32e+04     19.923      0.000    9.56e+05    1.16e+06
grade_9             1.029e+05   5004.876     20.568      0.000    9.31e+04    1.13e+05
grade_8             3.105e+04   3224.046      9.632      0.000    2.47e+04    3.74e+04
zipcode_98112       5.895e+05   1.09e+04     54.045      0.000    5.68e+05    6.11e+05
zipcode_98040       4.736e+05   1.03e+04     46.033      0.000    4.53e+05    4.94e+05
zipcode_98033       3.367e+05   8241.896     40.855      0.000    3.21e+05    3.53e+05
zipcode_98023      -5.471e+04   7859.177     -6.961      0.000   -7.01e+04   -3.93e+04
bathrooms_8.0       4.195e+06   1.57e+05     26.781      0.000    3.89e+06     4.5e+06
view_4.0            2.925e+05    1.2e+04     24.332      0.000    2.69e+05    3.16e+05
zipcode_98105        4.53e+05   1.14e+04     39.720      0.000    4.31e+05    4.75e+05
zipcode_98103       3.237e+05   7759.704     41.717      0.000    3.09e+05    3.39e+05
zipcode_98119       4.615e+05   1.29e+04     35.908      0.000    4.36e+05    4.87e+05
zipcode_98115       3.167e+05   7722.893     41.008      0.000    3.02e+05    3.32e+05
zipcode_98199       3.662e+05   1.01e+04     36.281      0.000    3.46e+05    3.86e+05
zipcode_98117       3.025e+05   7718.227     39.195      0.000    2.87e+05    3.18e+05
zipcode_98109       4.757e+05   1.62e+04     29.306      0.000    4.44e+05    5.07e+05
zipcode_98107       3.251e+05   1.06e+04     30.589      0.000    3.04e+05    3.46e+05
zipcode_98122       3.306e+05   1.04e+04     31.849      0.000     3.1e+05    3.51e+05
zipcode_98102       4.555e+05   1.63e+04     27.992      0.000    4.24e+05    4.87e+05
zipcode_98116       2.663e+05   9531.393     27.941      0.000    2.48e+05    2.85e+05
zipcode_98144       2.561e+05   9391.542     27.273      0.000    2.38e+05    2.75e+05
bathrooms_7.75      2.999e+06   1.57e+

In [ ]:
#Looking at scaling
test.hist(figsize=(7,10));

In [21]:
len(X_train_int.columns)

151